# Running ControlNet with Stable Diffusion XL

The goal here is to create an image given a text prompt and a base canny image (black background and white edge contours).

I'm having this problem: https://github.com/huggingface/diffusers/issues/4390#issuecomment-1676434622  
My PC is probably not good enough to run this.

https://huggingface.co/diffusers/controlnet-canny-sdxl-1.0  
https://huggingface.co/docs/diffusers/main/en/api/pipelines/controlnet  
https://huggingface.co/docs/diffusers/main/en/api/pipelines/controlnet_sdxl

In [57]:
%%capture
!apt-get update && apt-get install libgl1 -y
!pip install --upgrade diffusers[torch]
#!pip install transformers[torch]
!python -m pip install transformers[torch]
!pip install setuptools-rust
!pip install opencv-python

In [58]:
import transformers
print(transformers.__version__)
from diffusers import ControlNetModel, StableDiffusionControlNetPipeline, AutoencoderKL
from diffusers.utils import load_image
from PIL import Image
import torch
import numpy as np
import cv2

4.31.0


In [ ]:
prompt = "aerial view, a futuristic research complex in a bright foggy jungle, hard lighting"
negative_prompt = 'low quality, bad quality, sketches'

image = load_image("https://huggingface.co/datasets/hf-internal-testing/diffusers-images/resolve/main/sd_controlnet/hf-logo.png")

controlnet_conditioning_scale = 0.5  # recommended for good generalization

controlnet = ControlNetModel.from_pretrained(
    "diffusers/controlnet-canny-sdxl-1.0",
    torch_dtype=torch.float16
)
vae = AutoencoderKL.from_pretrained("madebyollin/sdxl-vae-fp16-fix", torch_dtype=torch.float16)
pipe = StableDiffusionControlNetPipeline.from_pretrained(
    "stabilityai/stable-diffusion-base-1.0",
    controlnet=controlnet,
    vae=vae,
    torch_dtype=torch.float16,
)
pipe.enable_model_cpu_offload()

image = np.array(image)
image = cv2.Canny(image, 100, 200)
image = image[:, :, None]
image = np.concatenate([image, image, image], axis=2)
image = Image.fromarray(image)

images = pipe(
    prompt, negative_prompt=negative_prompt, image=image, controlnet_conditioning_scale=controlnet_conditioning_scale,
    ).images

images[0].save(f"hug_lab.png")

In [ ]:
from diffusers import ControlNetModel, StableDiffusionControlNetPipeline

controlnet = ControlNetModel.from_pretrained("diffusers/controlnet-canny-sdxl-1.0")
pipeline = StableDiffusionControlNetPipeline.from_pretrained(
	"runwayml/stable-diffusion-v1-5", controlnet=controlnet
)